# Bivariate Hypothesis Tests

When conducting a cross-tab or a mean comparison test earlier in the semester, we were unable to say whether any difference in the dependent variable was "big enough". Well, hypothesis testing gives one way to make that determination. Is the observed relationship large enough that we are confident it is not a result of sampling error? Let's revisit our basic comparison tests from several weeks ago.

In [ ]:
# load necessary packages - install only if needed

#install.packages('pacman')

pacman::p_load(tidyverse, modelsummary)

# ingest survey data 
survey.df <- read.csv("https://raw.githubusercontent.com/bowendc/pol200_labs/refs/heads/main/simple_survey.csv")

If you recall, these data contain survey data on demographic characteristics of the survey respondents and whether (and degree) of interaction they have had with their state legislators.

## Hypothesis testing for cross-tabs with $\chi ^2$ tests

We can use the `chisq.test` function to conduct a $\chi ^2$ test. Just create your crosstab and store it as matrix in R; then call it up inside of `chisq.test()`. Notice in the block below that it uses frequencies, not column percentages.  

In [ ]:

# create the table with joint frequencies (counts)
# syntax: table(df$y, df$x)
# here we store the crosstab in the object matrix ct for future use
ct <- table(survey.df$cntrep_opinion, survey.df$income)
ct

# we can clean the code by rounding. 
round(prop.table(ct, 2), 2)

chisq.test(ct)


The function provides the test statistic, which is the calculated value of $\chi ^2$, then it calculates the probability we would get a value that large (given the degrees of freedom) by chance if the null were true. We could check the function's work by calculating the p-value ourselves:

In [ ]:
1 - pchisq(13.38, 3)

## Difference of means and proportions tests 

When conducting a mean comparison test, we can use a difference of means or difference of proportions test. The difference of means test, known as an *independent samples* t-test, is appropriate when the dependent variable is continuous. The difference of proportions tests is appropritate when the dependent variable is a 0,1 dichotomous variable. In both cases, our independent variable must be grouped into ***only*** two categories. In other words, you can only compare two means or proportions! In both tests, we need to process the dataframes a bit to get our data in the form these functions expect. 

### Difference of means

Use the `t.test()` function. There are several important arguments in this function. The defaults are to use the formula for unequal variances in your two groups, to conduct a two-tailed test, and to create a 95% confidence interval. All of these can be changed as needed, but will typically be what you will want to use. Note that the function accepts vectors of data for `x` and `y`, so below we use `filter()` and `select()` to grab just the data we need.

In [ ]:
# t.test expects a vector (single column) of data. Here,
#  we give two columns, one of y where women ==0 and one 
#  of y where women ==1. Need to use select() at end to 
#  have only a single vector from your dataframe

t.test(x = survey.df |> filter(women == 0) |> select(interactions), 
       y = survey.df |> filter(women == 1) |> select(interactions),
       mu = 0,            # test is that true diff is 0
       var.equal = FALSE, # does not assume equal variances; the default 
       conf.level = .95)  # default CL is .95 
                          # you could use different conf levels, like .90 or .99

### Difference of Sample Proportions

The `prop.test` function lets us test for the difference between two proportions. I think the easiest way to use this function is to first calculate the number of observations coded 1 on the dependent variable and the total number of observations, both grouped by the independent variable. We can do that using the standard `dplyr` functions and store for easy access. Then we call them back up inside of `prop.test()` 


In [ ]:
# quickly use dplyr syntax to get counts of successes (1s) out of all obs by IV
props <- survey.df |> 
              filter(!is.na(women) & !is.na(cntrep_opinions)) |> # filter out NAs
              group_by(women) |>                                 # group by IV
              summarize(successes = sum(cntrep_opinions),        # get number of successes 
                            n = n())                             # get total counts
props
prop.test(x = props$successes, 
          n = props$n,
       conf.level = .95)  # default CL is .95 
                          # you could use different conf levels, like .90 or .99
